In [1]:
# import libraries

import os
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import mygene
import sklearn.preprocessing as sk
import seaborn as sns
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from torch.utils.data.sampler import WeightedRandomSampler
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score, precision_recall_curve
import random
from random import randint
from sklearn.model_selection import StratifiedKFold
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [2]:
# define directories

cell_line_dir = "/Volumes/Expansion/Thesis Work/Supplementary Files/GDSC/"
models_dir = "/Volumes/Expansion/Thesis Work/Results/Transfer Learning/Drugs with same pathways/TCGA_CYTOi/Expression_Mutation/Models/"
DEGs_dir = "/Volumes/Expansion/Thesis Work/Results/GDSC_DEGs_inhibitors/CYTOi/"
dataset_dir = "/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/"
save_results_to = "/Volumes/Expansion/Thesis Work/Results/Transfer Learning/Drugs with same pathways/TCGA_CYTOi/Expression_Mutation/Predictions/"

In [3]:
# set random seeds

torch.manual_seed(42)
random.seed(42)

In [4]:
# change directory to read GDSC cell line details

os.chdir(cell_line_dir)
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Supplementary Files/GDSC


In [5]:
# read GDSC cell line details table

GDSC_cell_line_details = pd.read_excel("GDSC_Cell_Lines_Details.xlsx", keep_default_na = False)
GDSC_cell_line_details.set_index("COSMIC identifier", inplace = True)
GDSC_cell_line_details.tail()

,Sample Name,Whole Exome Sequencing (WES),Copy Number Alterations (CNA),Gene Expression,Methylation,Drug\nResponse,GDSC\nTissue descriptor 1,GDSC\nTissue\ndescriptor 2,Cancer Type\n(matching TCGA label),Microsatellite \ninstability Status (MSI),Screen Medium,Growth Properties
COSMIC identifier,,,,,,,,,,,,
1298218,KP-2,Y,N,Y,N,Y,pancreas,pancreas,PAAD,,D/F12,Adherent
1330932,KO52,Y,Y,N,N,N,leukemia,acute_myeloid_leukaemia,LAML,MSS/MSI-L,D/F12,NA
1331030,SC-1,Y,Y,N,N,N,lymphoma,B_cell_lymphoma,DLBC,MSS/MSI-L,R,NA
1503373,U-CH2,Y,Y,N,N,N,bone,bone_other,,MSS/MSI-L,D/F12,Adherent
,TOTAL:,1001,996,968,957,990,,,,,,


In [6]:
# change directory to read GDSC expression dataset (CYTOi)

os.chdir(dataset_dir + "/exprs/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/exprs


In [7]:
# read GDSC expression dataset (CYTOi)

GDSCE = pd.read_csv("GDSC_exprs.z.CYTOi.tsv", 
                    sep = "\t", index_col=0, decimal = ",")
GDSCE = pd.DataFrame.transpose(GDSCE)
GDSCE.head(3)

ENTREZID,1,2,9,10,12,13,14,15,16,18,...,107984199,107984208,107984325,107984411,107984648,107984889,107984989,107986313,107986782,107986810
683665,-0.4711563767951986,-0.1857522957766964,0.9702409548662462,-0.4081281805960821,-0.4161511169999803,-0.4384159723467665,-1.154692246005183,0.5262132484107975,-0.7950924576033422,-0.6337442716864214,...,1.283534690590172,-0.3367265464586691,-0.0039762988723447,0.7378729872604209,0.9757614264575688,0.1494417647095801,2.062294063911948,-0.8140636761719959,0.2870333828833719,0.9524265085537148
684055,1.3565140948247052,-0.277542411913307,0.0887087882196536,-0.2754829982302413,-0.5153856346587746,-0.6248712076433911,0.1243408990866293,-1.2699667684674136,2.0788360886382717,2.5197991997459197,...,-0.5571028630154293,-1.724090733574364,-0.3749080365068901,-0.0016055431935905,-0.3788535663547365,-1.3502092798315848,0.6949790577550328,0.4333032198982747,-0.1280986270308099,-1.629249958712223
684057,0.6510004960254864,1.6578876382433665,-0.4977106390881842,-0.1212172092933644,-0.398455281697677,-0.2804901106672752,0.6455898116420014,0.215833091774836,-0.1723595788875009,-0.2170870770552223,...,-0.024863133395904,0.4073439017665206,-0.4440356638831038,0.8630347078663075,1.4486171428489227,-1.4777214830771732,0.5220400480464272,0.6441211958947753,0.9057518606131394,0.3018783753479829


In [8]:
# change directory to read GDSC mutation dataset (CYTOi)

os.chdir(dataset_dir + "/mutations/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/mutations


In [9]:
# read GDSC mutation dataset (CYTOi)

GDSCM = pd.read_csv("GDSC_mutations.CYTOi.tsv",
                    sep="\t", index_col=0, decimal=".")
GDSCM.drop_duplicates(keep='last')
GDSCM = pd.DataFrame.transpose(GDSCM)
GDSCM = GDSCM.loc[:, ~GDSCM.columns.duplicated()]

GDSCM.head()

,143872,728577,2,2050,10243,79365,256006,4609,118788,113675,...,26517,28972,53917,283507,514,339210,64981,7335,5739,83442
683665,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
684055,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
684057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
684059,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
684062,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# change directory to read GDSC response dataset (CYTOi)

os.chdir(dataset_dir + "/response/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/response


In [11]:
# change directory to read GDSC response dataset (CYTOi)

GDSCR = pd.read_csv("GDSC_response.CYTOi.tsv",
                    sep="\t",
                    index_col=0,
                    decimal=",")
GDSCR.dropna(inplace=True)
GDSCR.rename(mapper=str, axis='index', inplace=True)
d = {"R": 0, "S": 1}
GDSCR["response"] = GDSCR.loc[:, "response"].apply(lambda x: d[x])

GDSCR.head()

,response,logIC50,drug,exprs,CNA,mutations
sample_name,,,,,,
683665,0,-3.64728727366553,Paclitaxel,1,1,1
683665,0,1.27553797434562,GSK269962A,1,1,1
683665,0,-4.54428601391955,Vinorelbine,1,1,1
683665,0,1.72219893512551,PF-562271,1,1,1
683665,0,1.70736111120553,IPA-3,1,1,1


In [12]:
# print count of cell lines for each drug (CYTOi)

GDSCR["drug"].value_counts()

drug
Vinorelbine     775
GSK429286A      771
Y-39983         769
Epothilone B    766
IPA-3           765
PF-562271       762
EHT 1864        761
Vinblastine     738
Docetaxel       738
Paclitaxel      370
GSK269962A      346
Name: count, dtype: int64

In [13]:
# change directory to read TCGA expression dataset homogenized with GDSC expression dataset (Docetaxel)

os.chdir(dataset_dir + "/exprs_homogenized/")
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/exprs_homogenized


In [14]:
# read TCGA expression dataset homogenized with GDSC expression dataset (Docetaxel)

TCGAEdocetaxel = pd.read_csv("TCGA_exprs.Docetaxel.eb_with.GDSC_exprs.Docetaxel.tsv",
                   sep = "\t", index_col=0, decimal = ",")
TCGAEdocetaxel = pd.DataFrame.transpose(TCGAEdocetaxel)
TCGAEdocetaxel.head(3)

ENTREZID,1,2,9,10,12,13,14,15,16,18,...,100188893,100190940,100190949,100191040,100192386,100268168,100271715,100287718,100288778,100289635
TCGA-50-5930-01A,4.32756185376123,5.27881192348218,6.03845362838735,3.07662245543196,3.97590092674197,4.34047859280406,7.72767194201385,2.67622920532167,8.90153773520786,3.7428901706813,...,9.39984480653599,2.91751762754054,2.99427874650319,3.46684192135117,2.95295666496376,2.56324156069538,2.83624184466872,3.30300759342689,3.83224472763852,3.96600205937326
TCGA-53-7624-01A,3.37553567415005,2.42891712354017,5.37898165978151,3.19267722666961,1.70915450915804,2.85662486974538,8.40128443433865,2.78716239748549,9.73072517371369,3.06801573314285,...,10.1613566288555,2.91751762754054,2.99427874650319,3.58970523529288,3.04238668392592,3.20663968789223,3.68240697177071,2.90828068411453,4.69456231162691,4.31887370809051
TCGA-A2-A0CK-01A,5.15835408802493,4.70501299235834,8.7884250410375,3.10059693730025,5.68150532486368,3.06721513901425,8.56654904306397,2.84369051122728,7.6379208249912,4.04190131728443,...,9.76797337512243,2.91751762754054,2.99427874650319,3.51280303534549,3.0066738528302,2.75549161640676,2.59081774047504,2.90828068411453,4.97889993433429,4.73966219270586


In [15]:
# change directory to read TCGA mutation dataset (Docetaxel)

os.chdir(dataset_dir + "mutations")
os.getcwd()

'/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/mutations'

In [16]:
# read TCGA mutation dataset (Docetaxel)

TCGAMdocetaxel = pd.read_csv("TCGA_mutations.Docetaxel.tsv",
                        sep="\t", index_col=0, decimal=",")
TCGAMdocetaxel.drop_duplicates(keep='last')
TCGAMdocetaxel = pd.DataFrame.transpose(TCGAMdocetaxel)
TCGAMdocetaxel = TCGAMdocetaxel.loc[:, ~TCGAMdocetaxel.columns.duplicated()]

TCGAMdocetaxel.head(3)

,1,2,3,9,10,12,13,14,15,16,...,106479642,106480474,106480731,106480736,107057644,107075310,107080636,107161144,109729113,109731405
TCGA-50-5930-01A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-53-7624-01A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-A2-A0CK-01A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# variance threshold for GDSC expression dataset (CYTOi)

selector = VarianceThreshold(0.05)
selector.fit_transform(GDSCE)
GDSCE = GDSCE[GDSCE.columns[selector.get_support(indices=True)]]

In [18]:
# fill NA values and binarize GDSC mutation dataset (CYTOi)

GDSCM = GDSCM.fillna(0)
GDSCM[GDSCM != 0.0] = 1

In [19]:
# select shared genes between GDSC and TCGA expression and mutation datasets 

ls = GDSCE.columns.intersection(GDSCM.columns)
ls = ls.intersection(TCGAEdocetaxel.columns)
ls = ls.intersection(TCGAMdocetaxel.columns)

ls = pd.unique(ls)

print(f"GDSC and TCGA # of common genes: {ls.shape[0]}")

GDSC and TCGA # of common genes: 17283


In [20]:
# select shared samples between GDSC expression, mutation, and response datasets (CYTOi)

ls2 = GDSCE.index.intersection(GDSCM.index)
ls2 = ls2.intersection(GDSCR.index)

print(f"GDSC # of common cell lines: {ls2.shape[0]}")

GDSC # of common cell lines: 801


In [21]:
# subset shared genes and samples in GDSC expression, mutation, and response datasets

GDSCE = GDSCE.loc[ls2, ls]
GDSCM = GDSCM.loc[ls2, ls]
GDSCR = GDSCR.loc[ls2, :]

print(f"GDSC # of common samples and genes (Expression): {GDSCE.shape}")
print(f"GDSC # of common samples and genes (Mutation): {GDSCM.shape}")
print(f"GDSC # of samples (Response): {GDSCR.shape[0]}")

GDSC # of common samples and genes (Expression): (801, 17283)
GDSC # of common samples and genes (Mutation): (801, 17283)
GDSC # of samples (Response): 7561


In [22]:
# select shared samples between TCGA expression and mutation datasets (Docetaxel) 

ls3 = TCGAEdocetaxel.index.intersection(TCGAMdocetaxel.index)

print(f"TCGA # of common samples (Docetaxel): {ls3.shape[0]}")

TCGA # of common samples (Docetaxel): 16


In [23]:
# select shared genes and samples between TCGA expression and mutation datasets (Docetaxel)

TCGAEdocetaxel = TCGAEdocetaxel.loc[ls3, ls]
TCGAMdocetaxel = TCGAMdocetaxel.loc[ls3, ls]

print(f"TCGA # of common samples and genes for Expression (Docetaxel): {TCGAEdocetaxel.shape}")
print(f"TCGA # of common samples and genes for Mutation (Docetaxel): {TCGAMdocetaxel.shape}\n")

TCGA # of common samples and genes for Expression (Docetaxel): (16, 17283)
TCGA # of common samples and genes for Mutation (Docetaxel): (16, 17283)



In [24]:
# change directory to read DEGs (CYTOi)

os.chdir(DEGs_dir)
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/GDSC_DEGs_inhibitors/CYTOi


In [25]:
# read DEGs (CYTOi)

DEGs_filtered_data = pd.read_excel("CYTOi_Differentially_Expressed_Genes (EnsemblID).xlsx",
                                    sheet_name = "Common DEGs")
print(f"There are {DEGs_filtered_data.shape[0]} DEGs (p-adjusted < 0.05 and |logFC| > 1) among cytoskeleton inhibitors\n")

DEGs_filtered_data.head()

There are 845 DEGs (p-adjusted < 0.05 and |logFC| > 1) among cytoskeleton inhibitors



,Gene Symbol,Frequency
0,PTGR1,8
1,SDC4,8
2,TPD52L1,8
3,S100A16,8
4,MYOF,8


In [26]:
# get gene symbol from mygene database

mg = mygene.MyGeneInfo()
DEGs_entrez_id = mg.querymany(DEGs_filtered_data["Gene Symbol"], 
                              species="human", 
                              scopes = "symbol",
                              field = "entrezgene",
                              as_dataframe=True)["entrezgene"]
DEGs_entrez_id = pd.unique(DEGs_entrez_id.dropna())
DEGs_entrez_id = pd.Series(DEGs_entrez_id)

result = mg.query("SLC22A18", species="human", scopes="symbol", fields="entrezgene")
DEGs_entrez_id = pd.concat([DEGs_entrez_id, pd.Series(str(result["hits"][1]["entrezgene"]))], ignore_index=True)

DEGs_entrez_id.shape[0]

2 input query terms found dup hits:	[('CLECL1P', 2), ('CAST', 2)]
1 input query terms found no hit:	['SLC22A18']


845

In [27]:
# assign GDSC data to new variables

exprs_z = GDSCE
mut = GDSCM
responses = GDSCR

In [28]:
# GDSC response dataset

responses.head(3)

,response,logIC50,drug,exprs,CNA,mutations
683665,0,-3.64728727366553,Paclitaxel,1,1,1
683665,0,1.27553797434562,GSK269962A,1,1,1
683665,0,-4.54428601391955,Vinorelbine,1,1,1


In [29]:
# list drug names in GDSC response dataset

drugs = set(responses["drug"].values)
drugs

{'Docetaxel',
 'EHT 1864',
 'Epothilone B',
 'GSK269962A',
 'GSK429286A',
 'IPA-3',
 'PF-562271',
 'Paclitaxel',
 'Vinblastine',
 'Vinorelbine',
 'Y-39983'}

In [30]:
# convert Entrez gene IDs to integer

responses.index = responses.index.astype(int)
responses.index

Index([ 683665,  683665,  683665,  683665,  683665,  683665,  683665,  683665,
        683665,  683665,
       ...
       1299050, 1299050, 1299050, 1299050, 1299050,  910941,  910941, 1659818,
       1659818, 1659818],
      dtype='int64', length=7561)

In [31]:
# merge GDSC cell line details with response dataset

dat = pd.merge(GDSC_cell_line_details,
         responses,
         left_index = True, 
         right_index=True,
         how = "inner")

dat["GDSC\nTissue descriptor 1"].value_counts()
#dat.head()

GDSC\nTissue descriptor 1
lung_NSCLC           800
urogenital_system    717
leukemia             703
lymphoma             642
aero_dig_tract       571
nervous_system       491
lung_SCLC            467
skin                 448
breast               405
digestive_system     367
large_intestine      355
bone                 340
neuroblastoma        281
pancreas             227
kidney               184
soft_tissue          157
myeloma              150
lung                 137
thyroid              119
Name: count, dtype: int64

In [32]:
# list count of cell lines as to tissues treated with docetaxel

filter = (dat["drug"] == "Docetaxel")
dat.loc[filter,"GDSC\nTissue descriptor 1"].value_counts()

GDSC\nTissue descriptor 1
lung_NSCLC           82
urogenital_system    74
leukemia             63
lymphoma             58
aero_dig_tract       58
nervous_system       48
lung_SCLC            44
skin                 42
breast               41
digestive_system     38
large_intestine      36
bone                 32
neuroblastoma        26
pancreas             24
kidney               17
myeloma              16
soft_tissue          14
lung                 13
thyroid              12
Name: count, dtype: int64

In [33]:
# get tissue descriptors

responses.index = responses.index.astype(str)
responses.index

Index(['683665', '683665', '683665', '683665', '683665', '683665', '683665',
       '683665', '683665', '683665',
       ...
       '1299050', '1299050', '1299050', '1299050', '1299050', '910941',
       '910941', '1659818', '1659818', '1659818'],
      dtype='object', length=7561)

In [34]:
# filter GDSC expression and mutation datasets (CYTOi) as to drugs
# subset selected DEGs

expression_zscores = []
mutations = []
for drug in drugs:
    samples = responses.loc[responses["drug"] == drug, :].index.values
    e_z = exprs_z.loc[samples, :]
    m = mut.loc[samples, :]
    expression_zscores.append(e_z)
    mutations.append(m)

GDSCEv2 = pd.concat(expression_zscores, axis=0)
GDSCMv2 = pd.concat(mutations, axis=0)
GDSCRv2 = responses

ls4 = list(set(GDSCE.columns).intersection(set(DEGs_entrez_id.astype(int))))
ls5 = GDSCEv2.index.intersection(GDSCMv2.index)

GDSCEv2 = GDSCEv2.loc[ls5, ls4]
GDSCMv2 = GDSCMv2.loc[ls5, ls4]
GDSCRv2 = GDSCRv2.loc[ls5, :]

TCGAEdocetaxel = TCGAEdocetaxel.loc[:,ls4]
TCGAMdocetaxel = TCGAMdocetaxel.loc[:,ls4]

responses.index = responses.index.values + "_" + responses["drug"].values

print(f"GDSC # of common samples and genes (Expression): {GDSCEv2.shape}")
print(f"GDSC # of common samples and genes (Mutation): {GDSCMv2.shape}")
print(f"GDSC # of common samples (Response): {GDSCRv2.shape[0]}\n")

print(f"TCGA # of common samples and genes for Docetaxel (Expression): {TCGAEdocetaxel.shape}")
print(f"TCGA # of common samples and genes for Docetaxel (Mutation): {TCGAMdocetaxel.shape}\n")

GDSC # of common samples and genes (Expression): (7561, 824)
GDSC # of common samples and genes (Mutation): (7561, 824)
GDSC # of common samples (Response): 7561

TCGA # of common samples and genes for Docetaxel (Expression): (16, 824)
TCGA # of common samples and genes for Docetaxel (Mutation): (16, 824)



In [35]:
# GDSC response values

Y = GDSCRv2['response'].values
print(Y)

[0 0 0 ... 1 0 0]


In [36]:
# change directory to read TCGA response dataset (Docetaxel)

os.chdir(dataset_dir + "response")
os.getcwd()

'/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/response'

In [37]:
# read TCGA response dataset (Docetaxel)

TCGARdocetaxel = pd.read_csv("TCGA_response.Docetaxel.tsv", 
                      sep = "\t", 
                      index_col=0, 
                      decimal = ",")
TCGARdocetaxel.dropna(inplace=True)
TCGARdocetaxel.rename(mapper=str, axis='index', inplace=True)
d = {"R": 0, "S": 1}
TCGARdocetaxel["response"] = TCGARdocetaxel.loc[:, "response"].apply(lambda x: d[x])
Ytsdocetaxel = TCGARdocetaxel["response"].values    

print(f"There are {Ytsdocetaxel.shape[0]} samples with response data in the TCGA (Docetaxel)\n")
Ytsdocetaxel

There are 16 samples with response data in the TCGA (Docetaxel)



array([0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1])

In [38]:
# TCGA cohort table with TCGA response dataset (Docetaxel)

dat_tcga = TCGARdocetaxel["cohort"]

dat_tcga.value_counts()

cohort
BRCA    8
LUAD    3
SARC    2
STAD    1
HNSC    1
BLCA    1
Name: count, dtype: int64

In [39]:
# filter BRCA cohort samples (Docetaxel)

filter = (TCGARdocetaxel["cohort"] == "BRCA")
tcga_brca_samples = dat_tcga.loc[filter,].index
tcga_brca_samples

Index(['TCGA-A2-A0CK-01A', 'TCGA-A2-A3XS-01A', 'TCGA-A2-A3XT-01A',
       'TCGA-A2-A3Y0-01A', 'TCGA-AC-A2FB-01A', 'TCGA-BH-A5IZ-01A',
       'TCGA-GM-A2DA-01A', 'TCGA-GM-A2DI-01A'],
      dtype='object', name='sample')

In [40]:
# filter responses of BRCA cohort samples (Docetaxel)

Ytsdocetaxel_brca = TCGARdocetaxel.loc[tcga_brca_samples,"response"].values
Ytsdocetaxel_brca

array([1, 0, 1, 1, 1, 1, 0, 1])

In [41]:
# count of sensitive and resistance BRCA samples (Docetaxel)

TCGARdocetaxel.loc[tcga_brca_samples,"response"].value_counts()

response
1    6
0    2
Name: count, dtype: int64

In [42]:
# define maximum iteration

max_iter = 50

In [51]:
# assign number of neurons and dropout rates found for layers from the saved models

hdm1 = 32
hdm2 = 256
rate1 = 0.7
rate2 = 0.6
rate3 = 0.6

In [52]:
# load pre-trained models and make predictions

scalerGDSC = sk.StandardScaler()
scalerGDSC.fit(GDSCEv2.values)
X_trainE = scalerGDSC.transform(GDSCEv2.values)
X_testEdocetaxel = scalerGDSC.transform(TCGAEdocetaxel.values)    

X_testEdocetaxel = scalerGDSC.transform(TCGAEdocetaxel.values)    
X_testEdocetaxel_brca = scalerGDSC.transform(TCGAEdocetaxel.loc[tcga_brca_samples,].values)    

X_trainM = np.nan_to_num(GDSCMv2.values)
X_testMdocetaxel = np.nan_to_num(TCGAMdocetaxel.values.astype(np.float32))
X_testMdocetaxel_brca = np.nan_to_num(TCGAMdocetaxel.loc[tcga_brca_samples,].values.astype(np.float32))

TX_testEdocetaxel = torch.FloatTensor(X_testEdocetaxel)
TX_testMdocetaxel = torch.FloatTensor(X_testMdocetaxel.astype(np.float32))
ty_testEdocetaxelorouracil = torch.FloatTensor(Ytsdocetaxel.astype(int))

TX_testEdocetaxel = torch.FloatTensor(X_testEdocetaxel)
TX_testEdocetaxel_brca = torch.FloatTensor(X_testEdocetaxel_brca)

TX_testMdocetaxel = torch.FloatTensor(X_testMdocetaxel.astype(np.float32))
TX_testMdocetaxel_brca = torch.FloatTensor(X_testMdocetaxel_brca.astype(np.float32))

n_sampE, IE_dim = X_trainE.shape
n_sampM, IM_dim = X_trainM.shape

h_dim1 = hdm1
h_dim2 = hdm2
Z_in = h_dim1 + h_dim2 

class AEE(nn.Module):
    def __init__(self):
        super(AEE, self).__init__()
        self.EnE = torch.nn.Sequential(
            nn.Linear(IE_dim, h_dim1),
            nn.BatchNorm1d(h_dim1),
            nn.ReLU(),
            nn.Dropout(rate1))
    def forward(self, x):
        output = self.EnE(x)
        return output  


class AEM(nn.Module):
    def __init__(self):
        super(AEM, self).__init__()
        self.EnM = torch.nn.Sequential(
            nn.Linear(IM_dim, h_dim2),
            nn.BatchNorm1d(h_dim2),
            nn.ReLU(),
            nn.Dropout(rate2))
    def forward(self, x):
        output = self.EnM(x)
        return output       

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.FC = torch.nn.Sequential(
            nn.Linear(Z_in, 1),
            nn.Dropout(rate3),
            nn.Sigmoid())
    def forward(self, x):
        return self.FC(x)

torch.cuda.manual_seed_all(42)

AutoencoderE = torch.load(models_dir + "Exprs_Docetaxel_GDSC_TCGA_CYTOi_Expression_Mutation.pt")
AutoencoderM = torch.load(models_dir + "Mut_Docetaxel_GDSC_TCGA_CYTOi_Expression_Mutation.pt")

Clas = torch.load(models_dir + "Class_Docetaxel_GDSC_TCGA_CYTOi_Expression_Mutation.pt")

AutoencoderE.eval()
AutoencoderM.eval()
Clas.eval()

ZEX = AutoencoderE(torch.FloatTensor(X_trainE))
ZMX = AutoencoderM(torch.FloatTensor(X_trainM))
ZTX = torch.cat((ZEX, ZMX), 1)
ZTX = F.normalize(ZTX, p=2, dim=0)
PredX = Clas(ZTX)
AUCT = roc_auc_score(Y, PredX.detach().numpy())
#print(f"GDSC - Training AUC (TCGA_CYTOi): {round(AUCT,2)}")
AUCTaucpr = average_precision_score(Y, PredX.detach().numpy())
#print(f"GDSC - Training AUCPR (TCGA_CYTOi): {round(AUCTaucpr,2)}\n")

ZETdocetaxel = AutoencoderE(TX_testEdocetaxel)
ZMTdocetaxel = AutoencoderM(TX_testMdocetaxel)
ZTTdocetaxel = torch.cat((ZETdocetaxel, ZMTdocetaxel), 1)
ZTTdocetaxel = F.normalize(ZTTdocetaxel, p=2, dim=0)
PredTdocetaxel = Clas(ZTTdocetaxel)
AUCTdocetaxel = roc_auc_score(Ytsdocetaxel.astype(int), PredTdocetaxel.detach().numpy())
#print(f"TCGA - Test AUC (Docetaxel): {round(AUCTdocetaxel,2)}")
AUCTdocetaxel_aucpr = average_precision_score(Ytsdocetaxel.astype(int), PredTdocetaxel.detach().numpy())
#print(f"TCGA - Test AUCPR (Docetaxel): {round(AUCTdocetaxel_aucpr,2)}\n")

ZETdocetaxel_brca = AutoencoderE(TX_testEdocetaxel_brca)
ZMTdocetaxel_brca = AutoencoderM(TX_testMdocetaxel_brca)
ZTTdocetaxel_brca = torch.cat((ZETdocetaxel_brca, ZMTdocetaxel_brca), 1)
ZTTdocetaxel_brca = F.normalize(ZTTdocetaxel_brca, p=2, dim=0)
PredTdocetaxel_brca = Clas(ZTTdocetaxel_brca)
AUCTdocetaxel_brca = roc_auc_score(Ytsdocetaxel_brca.astype(int), PredTdocetaxel_brca.detach().numpy())
#print(f"TCGA - Test AUC (Docetaxel, BRCA): {round(AUCTdocetaxel_brca,2)}")
AUCTdocetaxel_brca_aucpr = average_precision_score(Ytsdocetaxel_brca.astype(int), PredTdocetaxel_brca.detach().numpy())
#print(f"TCGA - Test AUCPR (Docetaxel, BRCA): {round(AUCTdocetaxel_brca_aucpr,2)}")

AUCT_finetuned = dict()
AUCTaucpr_finetuned = dict()
AUCTdocetaxel_finetuned = dict()
AUCTdocetaxel_brca_finetuned = dict()
AUCTdocetaxel_aucpr_finetuned = dict()
AUCTdocetaxel_brca_aucpr_finetuned = dict()
for iter in range(max_iter):
   iter += 1  

   AutoencoderE_finetuned = torch.load(models_dir + f"Finetuned_Models/Exprs_Docetaxel_GDSC_TCGA_CYTOi_Expression_Mutation_Finetuned_{iter}.pt")
   AutoencoderM_finetuned = torch.load(models_dir + f"Finetuned_Models/Mut_Docetaxel_GDSC_TCGA_CYTOi_Expression_Mutation_Finetuned_{iter}.pt")
   Clas_finetuned = torch.load(models_dir + f"Finetuned_Models/Class_Docetaxel_GDSC_TCGA_CYTOi_Expression_Mutation_Finetuned_{iter}.pt")
   
   AutoencoderE_finetuned.eval()
   AutoencoderM_finetuned.eval()
   Clas_finetuned.eval()
      
   ZEX = AutoencoderE_finetuned(torch.FloatTensor(X_trainE))
   ZMX = AutoencoderM_finetuned(torch.FloatTensor(X_trainM))
   ZTX = torch.cat((ZEX, ZMX), 1)
   ZTX = F.normalize(ZTX, p=2, dim=0)
   PredX = Clas_finetuned(ZTX)
   AUCT_finetuned[iter] = roc_auc_score(Y, PredX.detach().numpy())
   AUCTaucpr_finetuned[iter] = average_precision_score(Y, PredX.detach().numpy())
   
   ZETdocetaxel = AutoencoderE_finetuned(TX_testEdocetaxel)
   ZMTdocetaxel = AutoencoderM_finetuned(TX_testMdocetaxel)
   ZTTdocetaxel = torch.cat((ZETdocetaxel, ZMTdocetaxel), 1)
   ZTTdocetaxel = F.normalize(ZTTdocetaxel, p=2, dim=0)
   PredTdocetaxel = Clas_finetuned(ZTTdocetaxel)
   AUCTdocetaxel_finetuned[iter] = roc_auc_score(Ytsdocetaxel.astype(int), PredTdocetaxel.detach().numpy())
   AUCTdocetaxel_aucpr_finetuned[iter] = average_precision_score(Ytsdocetaxel.astype(int), PredTdocetaxel.detach().numpy())

   ZETdocetaxel_brca = AutoencoderE_finetuned(TX_testEdocetaxel_brca)
   ZMTdocetaxel_brca = AutoencoderM_finetuned(TX_testMdocetaxel_brca)
   ZTTdocetaxel_brca = torch.cat((ZETdocetaxel_brca, ZMTdocetaxel_brca), 1)
   ZTTdocetaxel_brca = F.normalize(ZTTdocetaxel_brca, p=2, dim=0)
   PredTdocetaxel_brca = Clas_finetuned(ZTTdocetaxel_brca)
   AUCTdocetaxel_brca_finetuned[iter] = roc_auc_score(Ytsdocetaxel_brca.astype(int), PredTdocetaxel_brca.detach().numpy())
   AUCTdocetaxel_brca_aucpr_finetuned[iter] = average_precision_score(Ytsdocetaxel_brca.astype(int), PredTdocetaxel_brca.detach().numpy())

max_key = max(AUCTdocetaxel_aucpr_finetuned, key=AUCTdocetaxel_aucpr_finetuned.get)
max_key_brca = max(AUCTdocetaxel_brca_aucpr_finetuned, key=AUCTdocetaxel_brca_aucpr_finetuned.get)

print(f"GDSC - Training AUC (CYTOi): {round(AUCT,2)}")
print(f"GDSC - Training AUCPR (CYTOi): {round(AUCTaucpr,2)}")
print(f"TCGA - Test AUC (Docetaxel): {round(AUCTdocetaxel,2)}")
print(f"TCGA - Test AUCPR (Docetaxel): {round(AUCTdocetaxel_aucpr,2)}")
print(f"TCGA - Test AUC (Docetaxel, BRCA): {round(AUCTdocetaxel_brca,2)}")
print(f"TCGA - Test AUCPR (Docetaxel, BRCA): {round(AUCTdocetaxel_brca_aucpr,2)}\n")

print(f"GDSC - Training AUC (CYTOi, Finetuned): {round(AUCT_finetuned[max_key],2)}")
print(f"GDSC - Training AUCPR (CYTOi, Finetuned): {round(AUCTaucpr_finetuned[max_key],2)}")
print(f"TCGA - Test AUC (Docetaxel, Finetuned): {round(AUCTdocetaxel_finetuned[max_key],2)}")
print(f"TCGA - Test AUCPR (Docetaxel, Finetuned): {round(AUCTdocetaxel_aucpr_finetuned[max_key],2)}\n")

print(f"GDSC - Training AUC (CYTOi, Finetuned): {round(AUCT_finetuned[max_key_brca],2)}")
print(f"GDSC - Training AUCPR (CYTOi, Finetuned): {round(AUCTaucpr_finetuned[max_key_brca],2)}")
print(f"TCGA - Test AUC (Docetaxel, BRCA, Finetuned): {round(AUCTdocetaxel_brca_finetuned[max_key_brca],2)}")
print(f"TCGA - Test AUCPR (Docetaxel, BRCA, Finetuned): {round(AUCTdocetaxel_brca_aucpr_finetuned[max_key_brca],2)}\n\n")

print("Models for maximum finetuning (Docetaxel)")
print(f"Exprs_Docetaxel_GDSC_TCGA_CYTOi_Expression_Mutation_Finetuned_{max_key}.pt")
print(f"Mut_Docetaxel_GDSC_TCGA_CYTOi_Expression_Mutation_Finetuned_{max_key}.pt")
print(f"Class_Docetaxel_GDSC_TCGA_CYTOi_Expression_Mutation_Finetuned_{max_key}.pt")

print("\nModels for maximum finetuning (Docetaxel, BRCA)")
print(f"Exprs_Docetaxel_GDSC_TCGA_CYTOi_Expression_Mutation_Finetuned_{max_key_brca}.pt")
print(f"Mut_Docetaxel_GDSC_TCGA_CYTOi_Expression_Mutation_Finetuned_{max_key_brca}.pt")
print(f"Class_Docetaxel_GDSC_TCGA_CYTOi_Expression_Mutation_Finetuned_{max_key_brca}.pt")

GDSC - Training AUC (CYTOi): 0.75
GDSC - Training AUCPR (CYTOi): 0.3
TCGA - Test AUC (Docetaxel): 0.56
TCGA - Test AUCPR (Docetaxel): 0.57
TCGA - Test AUC (Docetaxel, BRCA): 0.83
TCGA - Test AUCPR (Docetaxel, BRCA): 0.96

GDSC - Training AUC (CYTOi, Finetuned): 0.76
GDSC - Training AUCPR (CYTOi, Finetuned): 0.31
TCGA - Test AUC (Docetaxel, Finetuned): 0.7
TCGA - Test AUCPR (Docetaxel, Finetuned): 0.67

GDSC - Training AUC (CYTOi, Finetuned): 0.75
GDSC - Training AUCPR (CYTOi, Finetuned): 0.3
TCGA - Test AUC (Docetaxel, BRCA, Finetuned): 0.83
TCGA - Test AUCPR (Docetaxel, BRCA, Finetuned): 0.96


Models for maximum finetuning (Docetaxel)
Exprs_Docetaxel_GDSC_TCGA_CYTOi_Expression_Mutation_Finetuned_15.pt
Mut_Docetaxel_GDSC_TCGA_CYTOi_Expression_Mutation_Finetuned_15.pt
Class_Docetaxel_GDSC_TCGA_CYTOi_Expression_Mutation_Finetuned_15.pt

Models for maximum finetuning (Docetaxel, BRCA)
Exprs_Docetaxel_GDSC_TCGA_CYTOi_Expression_Mutation_Finetuned_8.pt
Mut_Docetaxel_GDSC_TCGA_CYTOi_Express

In [45]:
# create a dataframe with the prediction results 

em = {"Data": ["Expr + Mut", "Expr + Mut (Fine-tuned)"],
      "AUC (GDSC)": [round(AUCT, 2), round(AUCT_finetuned[max_key], 2)],
      "AUCPR (GDSC)": [round(AUCTaucpr,2), round(AUCTaucpr_finetuned[max_key],2)],
      "AUC (TCGA)": [round(AUCTdocetaxel,2), round(AUCTdocetaxel_finetuned[max_key],2)],
      "AUCPR (TCGA)":  [round(AUCTdocetaxel_aucpr,2), round(AUCTdocetaxel_aucpr_finetuned[max_key],2)],
      "AUC (TCGA-BRCA)":  [round(AUCTdocetaxel_brca,2), round(AUCTdocetaxel_brca_finetuned[max_key_brca],2)],
      "AUCPR (TCGA-BRCA)": [round(AUCTdocetaxel_brca_aucpr,2), round(AUCTdocetaxel_brca_aucpr_finetuned[max_key_brca],2)],
      "Sample Size (GDSC)": [GDSCEv2.shape[0], GDSCEv2.shape[0]],
      "Feature Size (GDSC)": [GDSCEv2.shape[1], GDSCEv2.shape[1]] 
}
em_dataframe = pd.DataFrame.from_dict(em, orient='index').transpose()  
em_dataframe.to_csv(save_results_to + "GDSC_TCGA_Expression_Mutation_CYTOi_Docetaxel.tsv",
                    sep = "\t",
                    index = False)

em_dataframe

,Data,AUC (GDSC),AUCPR (GDSC),AUC (TCGA),AUCPR (TCGA),AUC (TCGA-BRCA),AUCPR (TCGA-BRCA),Sample Size (GDSC),Feature Size (GDSC)
0,Expr + Mut,0.75,0.3,0.56,0.57,0.83,0.96,7561,824
1,Expr + Mut (Fine-tuned),0.76,0.31,0.7,0.67,0.83,0.96,7561,824


In [46]:
# show expression layer parameters

AutoencoderE

AEE(
  (EnE): Sequential(
    (0): Linear(in_features=824, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.7, inplace=False)
  )
)

In [47]:
# show mutation layer parameters

AutoencoderM

AEM(
  (EnM): Sequential(
    (0): Linear(in_features=824, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.6, inplace=False)
  )
)

In [48]:
# show classification layer parameters

Clas

Classifier(
  (FC): Sequential(
    (0): Linear(in_features=288, out_features=1, bias=True)
    (1): Dropout(p=0.6, inplace=False)
    (2): Sigmoid()
  )
)

In [49]:
# change directory

os.chdir(save_results_to)
print(os.getcwd())

/Volumes/Expansion/Thesis Work/Results/Transfer Learning/Drugs with same pathways/TCGA_CYTOi/Expression_Mutation/Predictions


In [50]:
# save predictions

file = open('GDSC_TCGA_DEGs - TCGA_CYTOi_Docetaxel Predictions (Expression and Mutation).txt', 'w')
file.write(f"GDSC Training (EM) AUC (TCGA_CYTOi): {round(AUCT,2)}\n")
file.write(f"GDSC Training (EM) AUCPR (TCGA_CYTOi): {round(AUCTaucpr,2)}\n\n")

file.write(f"TCGA Test (EM) AUC (Docetaxel): {round(AUCTdocetaxel,2)}\n")
file.write(f"TCGA Test (EM) AUCPR (Docetaxel): {round(AUCTdocetaxel_aucpr,2)}\n\n")

file.close()